In [106]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np
import random

In [107]:
df = pd.read_csv("covid_analytics_clinical_data.csv")
c_df = df[['Country','Study Pop Size (N)','Severity','Survivors']]
arr = c_df['Severity'].to_numpy()
arr = pd.unique(arr)
c_df["Severity"]=c_df["Severity"].replace(arr,[1,3,2,-1,1,2,3,2,3,4])
arr = c_df['Survivors'].to_numpy()
arr = pd.unique(arr)
c_df["Survivors"]=c_df["Survivors"].replace(arr,[0,1,2,-1])
arr = c_df['Country'].to_numpy()
arr = pd.unique(arr)
c_df["Country"]=c_df["Country"].replace(arr,['1', '2', '3', '4', '5',
       '6', '7', '8', '9',
       '10', '11', '12', '13', '14',
       '15', '16', '17', '18', '19',
       '20', -1])
c_df = c_df.fillna(-1)
c_df

<ipython-input-107-43ca2e11bf1b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  c_df["Severity"]=c_df["Severity"].replace(arr,[1,3,2,-1,1,2,3,2,3,4])
<ipython-input-107-43ca2e11bf1b>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  c_df["Survivors"]=c_df["Survivors"].replace(arr,[0,1,2,-1])
<ipython-input-107-43ca2e11bf1b>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

,Country,Study Pop Size (N),Severity,Survivors
0,1,191.0,1,0
1,1,54.0,3,1
2,1,137.0,1,2
3,1,199.0,1,0
4,1,99.0,1,0
...,...,...,...,...
534,3,99.0,1,-1
535,3,117.0,1,-1
536,3,247.0,1,-1
537,3,50.0,1,-1
